In [1]:
import requests
from bs4 import BeautifulSoup
import subprocess
import re
import os
import stanfordnlp
import pandas as pd

from ProcessArticle import ProcessArticle

from WikiArticleXmlHandler import WikiArticleXmlHandler
from WikiDataXmlHandler import WikiDataXmlHandler
import xml.sax
import mwparserfromhell

from timeit import default_timer as timer

from multiprocessing import Pool 
from multiprocessing import Lock
import tqdm 

# List of lists to single list
from itertools import chain

# Sending keyword arguments in map
from functools import partial

import gc

import warnings
warnings.filterwarnings('ignore')

In [2]:
infoboxes = pd.read_csv("./2.data/infobox-list.csv")["infobox template name"].values.tolist()

article_folder = "./0.articles"
articles = [article_folder + "/" + x for x in os.listdir(article_folder)]
article = articles[0] # process only one file for now

wikidata_folder = "./1.wikidata"
wikidatas = [wikidata_folder + "/" + x for x in os.listdir(wikidata_folder)]
wikidata = wikidatas[0]

out_file = "./saved/out_nlp_"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp)

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [ ]:
%%time

handler = WikiArticleXmlHandler(out_file + ".csv", func, None)
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(file))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(file),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()

In [3]:
if not os.path.exists(out_file + "_wikidata.csv"):
    pd.DataFrame(columns=["title", "label", "type"]).to_csv(
        out_file + "_wikidata.csv", header=True, index=False, sep="\t")

In [3]:
%%time

handler = WikiDataXmlHandler(out_file + "_wikidata.csv")
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(wikidata))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(wikidata),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()

Started processing: ./1.wikidata/wikidatawiki-latest-pages-articles-multistream1.xml-p1p235321.bz2
Saved to csv, current number of articles <1000>
Label key not found in Q1379
Saved to csv, current number of articles <2000>
Saved to csv, current number of articles <3000>
No english (en) available for Q4163
Saved to csv, current number of articles <4000>
Saved to csv, current number of articles <5000>
No P31, P279, P361 labels in: Q5299
Label key not found in Q5641


ValueError: arrays must all be same length